## Optuna Example Hyperparameter Optimization KNN, SVD, NNMF

In order for the code below to work for a different method/model, the model should have member functions like this:

    def get_hyper_params(self):
        hparams = {'num_factors': {'type': 'integer', 'values': [2, 10]},
                   'rho_1': {'type': 'loguniform', 'values': [1e-3, 100]},
                   'rho_2': {'type': 'loguniform', 'values': [1e-3, 100]}}
        return hparams

    def set_hyper_params(self, **kwargs):
        self.num_factors = kwargs['num_factors']
        self.rho1 = kwargs['rho_1']
        self.rho2 = kwargs['rho_2']

In [2]:
import os, sys
sys.path.append('../')
import hp_optimization as hopt
from optuna.visualization import plot_optimization_history, \
                                 plot_intermediate_values, \
                                 plot_contour

from design import ModelTraining
from methods.matrix_factorization.MF_STL import MF_STL
from methods.matrix_factorization.MF import SVD_MF, NonNegative_MF
from methods.knn.KNN import KNN_Normalized
from shutil import copyfile
from UTILS.utils import datasetParams2str
from datasets import SyntheticData as SD

outdir = '../outputs/experiment_004x' # make sure that it lines up with the experiment's filename
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [3]:
dataset = SD.SyntheticDataCreator(num_tasks=3,cellsPerTask=400, drugsPerTask=10, function="cosine",
             normalize=False, noise=1, graph=False, test_split=0.3)
dataset.prepare_data()

In [ ]:
methods  = [KNN_Normalized(k=10), SVD_MF(n_factors=100), NonNegative_MF(n_factors=100)]
i = 0 
for method in methods:
    if i == 0:
        study = hopt.optimize_hyper_params(method, dataset,n_trials=5)
        i += 1
    else:
        study = hopt.optimize_hyper_params(method, dataset,n_trials=50)
    plot_optimization_history(study)
    plot_intermediate_values(study)
    plot_contour(study)
    print("best params for "+ method.name + " : ",study.best_params)
    # copy the study, i.e. hyperparam trials
    dataset_str = datasetParams2str(dataset.__dict__)
    study_name = '{}_{}'.format(method.name,dataset_str)
    storage='hyperparam_experiments/{}.db'.format(study_name)
    copyfile(storage, os.path.join(outdir,study_name + '.db'))
    